In [41]:
import os
import glob
import random
import pandas as pd
import numpy as np

# Získanie datasetu

In [49]:
# Download dataset
DATASET_URL = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
DATASET_DIR = 'dataset'
DATASET_FILE_PATH = f'{DATASET_DIR}/aclImdb_v1.tar.gz'

if not os.path.isfile(DATASET_FILE_PATH):
    print(f'Downloading dataset into {DATASET_FILE_PATH} ...')
    with urllib.request.urlopen(DATASET_URL) as response, open(DATASET_FILE_PATH, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
else:
    print('Dataset already downloaded.')

Dataset already downloaded.


In [50]:
# Untar the dataset archive
if not os.path.isdir(f'{DATASET_DIR}/aclImdb'):
    with tarfile.open(DATASET_FILE_PATH) as archive:
        print(f'Extracting "{DATASET_FILE_PATH}" to "{DATASET_DIR}" ...')
        archive.extractall(DATASET_DIR)
        print('Extraction finished.')
else:
    print('Dataset already extracted.')


Dataset already extracted.


In [51]:
# Load data from folders
TEST_FOLDER = f'{DATASET_DIR}/aclImdb/test'
TEST_POSITIVE_FOLDER = f'{TEST_FOLDER}/pos'
TEST_NEGATIVE_FOLDER = f'{TEST_FOLDER}/neg'

TRAIN_FOLDER = f'{DATASET_DIR}/aclImdb/train'
TRAIN_POSITIVE_FOLDER = f'{TRAIN_FOLDER}/pos'
TRAIN_NEGATIVE_FOLDER = f'{TRAIN_FOLDER}/neg'
TRAIN_UNSUPERVISED_FOLDER = f'{TRAIN_FOLDER}/unsup'

VOCAB_SIZE = 10_000
MAX_SENTENCE_LEN = 100

In [52]:
TEST_FOLDER

'dataset/aclImdb/test'

In [53]:
def create_labeled_dataset_from_files(folders, label_map={'pos':[1, 0], 'neg': [0, 1]}, shuffle=True):
    files = map(lambda folder: [glob.glob(f'{folder}/*'), f'{folder}'], folders)

    # Assign label to every files based on folder they are in
    labeled_files = map(lambda files_with_label:
                        map(lambda file_path:
                            [file_path, files_with_label[1].split('/')[-1]] # Take only the last folde from the folder path
                        , files_with_label[0])
                    , files)

    # flatten list
    flat_labeled_files = []
    for lf in labeled_files:
        for fl in lf:
            flat_labeled_files.append(fl)

    if shuffle:
        random.shuffle(flat_labeled_files)

    # read file contents
    labeled_texts = map(lambda example: [open(example[0]).read().split(' ')[:MAX_SENTENCE_LEN], example[1]], flat_labeled_files)

    return labeled_texts

cls_test_ds = create_labeled_dataset_from_files([f'{TEST_POSITIVE_FOLDER}', f'{TEST_NEGATIVE_FOLDER}'])
cls_train_ds = create_labeled_dataset_from_files([f'{TRAIN_POSITIVE_FOLDER}', f'{TRAIN_NEGATIVE_FOLDER}'])

In [54]:
reviews = []
for review in list(cls_test_ds):
    reviews.append([" ".join(review[0]), review[1]])

In [55]:
test_df = pd.DataFrame(np.array(reviews), columns=['review_text', 'sentiment'])

# Ukážka datasetu

Dataset pozostáva z dvoch atribútov - text recenzie a sentiment. Oba atribúty sú v textovej podobe, čiže je nutné ich upraviť do číselnej podoby.

In [56]:
test_df.head()

,review_text,sentiment
0,Good grief.. to think I've seen it all.. Danny...,neg
1,"Wow, this movie is bad. Think ""Flashdance"" wit...",neg
2,This movie was Jerry Bruckheimer's idea to sel...,neg
3,`Skenbart' is one of the funniest movies to no...,pos
4,"I saw this at The Tribeca Film Festival, in th...",pos


In [57]:
test_df.describe()

,review_text,sentiment
count,25000,25000
unique,24795,2
top,Loved today's show!!! It was a variety and not...,neg
freq,5,12500


Z nasledujúcích údajov môžeme vidieť, že v datasete sa nachádza 24795 unikátnych recenzií, respektíve 225 duplikátov.

In [61]:
test_df.dtypes

review_text    object
sentiment      object
dtype: object

# Prekryv prvkov z train a test datasetu

In [ ]:
TODO

# min, mod, median -> dlzky reviews v pos a neg.


In [ ]:
TODO